# Import Dependencies

In [53]:
import os
import cv2

import optuna
import insightface
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import umap.umap_ as umap

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from tqdm import tqdm
from insightface.app import FaceAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import StackingClassifier, VotingClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
CONFIG = {
    "APP_MODEL": "buffalo_l",
    "APP_CONFIG": {
        "ctx_id": 0,
        "det_size": (320, 320),
        "det_thresh": 0.4,
    },
    "RANDOM_STATE": 42,
    "FEATURE_WEIGHTS": {
        "embeddings": 1.0,
        "norm_bbox": 0.5,
    },
    "ENHANCE_EMB": True
}

In [3]:
main_dir = "dataset"

train_dir = os.path.join(main_dir, "train")
train_labels_dir = os.path.join(train_dir, "labels.csv")
test_dir = os.path.join(main_dir, "test")
ref_dir = os.path.join(main_dir, "reference_faces")

In [4]:
train_labels = pd.read_csv(train_labels_dir)
train_labels.head()

,filename,emp_id
0,face_0568.jpg,emp016
1,face_0433.jpg,emp014
2,face_1751.jpg,emp004
3,face_0675.jpg,emp028
4,face_0112.jpg,emp001


# Create Reference Data

In [5]:
app = FaceAnalysis(name=CONFIG["APP_MODEL"])
app.prepare(**CONFIG["APP_CONFIG"])

def robust_face_detection(img, app, attempts=3):
    if img is None:
        return None
        
    strategies = [
        lambda x: x,  # Original
        lambda x: cv2.convertScaleAbs(x, alpha=1.5, beta=40),  # Brighten
        lambda x: cv2.equalizeHist(cv2.cvtColor(x, cv2.COLOR_BGR2GRAY))[:,:,np.newaxis].repeat(3,2),  # Hist equal
        lambda x: cv2.GaussianBlur(x, (5,5), 0),  # De-noise
        lambda x: cv2.medianBlur(x, 3),  # Alternative de-noise
    ]
    
    for i in range(attempts):
        try:
            modified = strategies[i](img) if i < len(strategies) else img
            faces = app.get(modified)
            if len(faces) > 0:
                return faces
        except Exception as e:
            print(f"Detection attempt {i+1} failed: {str(e)}")
            continue
    return None

/home/seang/miniconda3/envs/DataMind/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:121: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/seang/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/seang/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/seang/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/seang/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/seang/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det

In [6]:
import numpy as np

def enhance_embedding(
    embedding, 
    face, 
    landmark_weight=0.12,
    pose_weight=0.08,
    kps_weight=0.05,
    det_score_weight=0.04,
):
    required_attrs = ['bbox', 'kps', 'det_score', 'landmark_2d_106', 'pose']
    missing = [attr for attr in required_attrs if not hasattr(face, attr)]
    if missing:
        raise ValueError(f"Missing required face attributes: {missing}")
    
    base_embed = embedding / (np.linalg.norm(embedding))

    bbox_w = face.bbox[2] - face.bbox[0]
    bbox_h = face.bbox[3] - face.bbox[1]
    if bbox_w <= 0 or bbox_h <= 0:
        raise ValueError("Invalid bounding box dimensions.")
    
    landmarks_rel = ((face.landmark_2d_106 - face.bbox[:2]) / [bbox_w, bbox_h]).flatten()
    kps_rel = ((face.kps - face.bbox[:2]) / [bbox_w, bbox_h]).flatten()

    yaw, pitch, roll = face.pose
    yaw_rad = np.deg2rad(yaw)
    pitch_rad = np.deg2rad(pitch)
    roll_rad = np.deg2rad(roll)
    
    norm_pose = np.array([
        np.clip(yaw_rad / np.pi, -1, 1),
        np.clip(pitch_rad / (np.pi / 2), -1, 1),
        np.clip(roll_rad / np.pi, -1, 1)
    ])
    
    enhanced = np.concatenate([
        base_embed,
        landmark_weight * landmarks_rel,
        kps_weight * kps_rel,
        pose_weight * norm_pose,
        det_score_weight * np.array([face.det_score]),
    ])
    
    return enhanced / (np.linalg.norm(enhanced))


In [7]:
def create_reference_data(ref_dir, ref_labels, app):
    ref_data = {
        "embedding": [],
        "enhanced_embedding": [],
        "emp_id": [],
        "norm_bbox": [],
        "is_ref": []
    }
    
    for emp_id in tqdm(ref_labels, desc="Processing Reference Identities"):
        img_path = os.path.join(ref_dir, emp_id)
        if os.path.isdir(img_path) is False:
            print(f"Error processing {emp_id} folder: Is not directory")
            continue
        for filename in tqdm(os.listdir(img_path), desc=f"Processing {emp_id} folder"):
            filepath = os.path.join(img_path, filename)
            if filepath.endswith(".mp4"):
                continue
            img = cv2.imread(filepath)
            faces = robust_face_detection(img, app, attempts=4)
            if faces:
                face = faces[0]
                img_h, img_w = img.shape[:2]
                bbox = face.bbox
                norm_bbox = [
                    bbox[0]/img_w,
                    bbox[1]/img_h,
                    bbox[2]/img_w,
                    bbox[3]/img_h
                ]
                ref_data['embedding'].append(
                    (face.embedding / np.linalg.norm(face.embedding))
                )
                ref_data['enhanced_embedding'].append(
                    enhance_embedding(face.embedding, face)
                )
                ref_data['emp_id'].append(emp_id)
                ref_data['norm_bbox'].append(norm_bbox)
            else:
                print(f"Error: Could not detect any faces in {filepath}")
            ref_data["is_ref"].append(1)
    
    for key in ref_data:
        if key == 'emp_id':
            continue
        ref_data[key] = np.array(ref_data[key])

    return ref_data

ref_data = create_reference_data(ref_dir, sorted(os.listdir(ref_dir)), app)
ref_data["embedding"].shape, ref_data["enhanced_embedding"].shape

Processing Reference Identities:   0%|          | 0/34 [00:00<?, ?it/s]/home/seang/miniconda3/envs/DataMind/lib/python3.10/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4
Processing Reference Identities:  21%|██        | 7/34 [00:25<01:41,  3.75s/it]

Error: Could not detect any faces in dataset/reference_faces/emp008/emp008_013.jpg


Processing Reference Identities:  38%|███▊      | 13/34 [00:53<01:34,  4.48s/it]

Error: Could not detect any faces in dataset/reference_faces/emp013/emp013_014.jpg


Processing Reference Identities:  85%|████████▌ | 29/34 [02:15<00:25,  5.05s/it]

Error: Could not detect any faces in dataset/reference_faces/emp030/emp030_011.jpg


Error: Could not detect any faces in dataset/reference_faces/emp030/emp030_003.jpg


Processing Reference Identities: 100%|██████████| 34/34 [02:38<00:00,  4.66s/it]


((448, 512), (448, 738))

# Create Train Data

In [8]:
def create_train_data(train_dir, labels_df, app, normalize=True):
    train_data = {
        "embedding": [],
        "enhanced_embedding": [],
        "emp_id": [],
        "norm_bbox": [],
        "is_ref": [],
    }
    failed_samples = []

    img_path = os.path.join(train_dir, "images")
    for idx, row in tqdm(labels_df.iterrows(), desc="Processing Train Identities", total=len(labels_df)):
        filename, emp_id = row
        filepath = os.path.join(img_path, filename)
        
        img = cv2.imread(filepath)
        faces = robust_face_detection(img, app, attempts=4)
        
        if faces:
            face = faces[0]
            img_h, img_w = img.shape[:2]
            bbox = face.bbox
            norm_bbox = [
                bbox[0]/img_w,
                bbox[1]/img_h,
                bbox[2]/img_w,
                bbox[3]/img_h
            ]
            train_data['embedding'].append(
                (face.embedding / np.linalg.norm(face.embedding))
            )
            train_data['enhanced_embedding'].append(
                enhance_embedding(face.embedding, face)
            )
            train_data['emp_id'].append(emp_id)
            train_data['norm_bbox'].append(norm_bbox)
        else:
            # train_data['embedding'].append(np.random.normal(0, 0.01, 512))
            # train_data['enhanced_embedding'].append(np.random.normal(0, 0.01,738))
            # train_data['emp_id'].append("UNKNOWN")
            # train_data['norm_bbox'].append([0.0, 0.0, 0.0, 0.0])
            failed_samples.append((emp_id, filepath))
        
        train_data["is_ref"].append(0)
    
    for key in train_data:
        if key == 'emp_id':
            continue
        train_data[key] = np.array(train_data[key])
    
    return train_data, failed_samples

train_data, train_failed = create_train_data(train_dir, train_labels, app)

Processing Train Identities: 100%|██████████| 1179/1179 [07:02<00:00,  2.79it/s]


In [17]:
train_data["embedding"].shape, train_data["enhanced_embedding"].shape

((1066, 512), (1066, 738))

In [16]:
print(f"Training images used (%): {((len(train_labels) - len(train_failed)) / len(train_labels)) * 100:.2f}")

Training images used (%): 90.42


In [18]:
scaler = StandardScaler()

def create_feature_vectors(ref_data, scaler, return_target=True, weights=None, enhanced_embedding=False):

    if enhanced_embedding:
        embeddings = np.array(ref_data["enhanced_embedding"])
    else:
        embeddings = np.array(ref_data["embedding"])
    norm_bbox = np.array(ref_data["norm_bbox"])

    features_to_normalize = np.hstack([
        norm_bbox
    ])

    normalized_features = scaler.fit_transform(features_to_normalize)
    
    norm_bbox = normalized_features[:, :4]

    features = np.hstack([
        embeddings * weights["embeddings"],        
        norm_bbox * weights["norm_bbox"],
    ])
    
    metadata = {
        "emp_ids": np.array(ref_data["emp_id"]) if return_target else None,
        "feature_names": [
            *[f"embedding_{i}" for i in range(512)],
            "bbox_x1", "bbox_y1", "bbox_x2", "bbox_y2",
        ],
        "scaler": scaler
    }
    if return_target:
        target = ref_data['emp_id']
    else:
        target = None

    return features, target, metadata

In [19]:
ref_feats, ref_target, ref_metadata = create_feature_vectors(
    ref_data, scaler, weights=CONFIG["FEATURE_WEIGHTS"], enhanced_embedding=CONFIG["ENHANCE_EMB"]
)
ref_feats.shape

(448, 742)

In [20]:
train_feats, train_target, train_metadata = create_feature_vectors(
    train_data, scaler, weights=CONFIG["FEATURE_WEIGHTS"], enhanced_embedding=CONFIG["ENHANCE_EMB"]
)
train_feats.shape

(1066, 742)

# Get Features and Labels

In [21]:
classes = np.unique(train_target + ref_target)
le = LabelEncoder()
le.fit(classes)

X = np.vstack([train_feats, ref_feats])
y = np.hstack([le.transform(train_target), le.transform(ref_target)])

In [22]:
def plot_umap_with_refs(features, labels, is_ref, title="UMAP Projection with Reference Highlight"):
    reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=CONFIG["RANDOM_STATE"])
    emb_2d = reducer.fit_transform(features)
    
    plt.figure(figsize=(10, 8))
    ref_mask = is_ref == 1
    train_mask = is_ref == 0
    
    plt.scatter(emb_2d[train_mask, 0], emb_2d[train_mask, 1], c=labels[train_mask], cmap="tab20", s=10, alpha=0.5, label="Train")
    plt.scatter(emb_2d[ref_mask, 0], emb_2d[ref_mask, 1], edgecolors='k', c=labels[ref_mask], cmap="tab20", s=30, alpha=0.9, label="Reference", marker='x')
    
    plt.title(title)
    plt.grid(True)
    plt.legend()
    plt.show()

plot_umap_with_refs(X, y, np.concatenate((train_data['is_ref'] ,ref_data['is_ref'])))

/home/seang/miniconda3/envs/DataMind/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/seang/miniconda3/envs/DataMind/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


IndexError: boolean index did not match indexed array along dimension 0; dimension is 1514 but corresponding boolean dimension is 1631

<Figure size 1000x800 with 0 Axes>

In [46]:
from sklearn.base import BaseEstimator, ClassifierMixin

class CosineSimilarityModel(BaseEstimator, ClassifierMixin):
    def __init__(self, threshold=0.6, unknown_label="UNKNOWN"):
        self.threshold = threshold
        self.unknown_label = unknown_label
        self.classes_ = None
        self.prototypes_ = None
        
    def fit(self, X, y):
        self.classes_ = np.unique(y)
        # Store both string and encoded labels
        self.le_ = LabelEncoder().fit(y)
        self.prototypes_ = np.array([X[y == cls].mean(axis=0) for cls in self.classes_])
        print(self.prototypes_.shape)
        return self
        
    def predict(self, X):
        scores = cosine_similarity(X, self.prototypes_)
        best_scores = np.max(scores, axis=1)
        best_indices = np.argmax(scores, axis=1)
        
        # Convert to numerical labels first
        num_preds = np.where(
            best_scores >= self.threshold,
            self.le_.transform(self.classes_[best_indices]),
            -1  # Temporary unknown code
        )
        
        return num_preds
    
    def predict_proba(self, X):
        scores = cosine_similarity(X, self.prototypes_)
        exp_scores = np.exp(scores - np.max(scores, axis=1, keepdims=True))
        return exp_scores / exp_scores.sum(axis=1, keepdims=True)

In [24]:
class CrossValidator:
    def __init__(self, models, metric_fns, cv_method, name=None, verbose=True):
        self.models = models
        self.metric_fns = metric_fns
        self.cv_method = cv_method
        self.name = name
        self.verbose = verbose
        self.results = {}

    def _calculate_metrics(self, y_true, y_pred):
        results = {}
        for name, fn in self.metric_fns:
            try:
                if name == "macro_f1":
                    results[name] = fn(y_true, y_pred, average="macro")
                else:
                    results[name] = fn(y_true, y_pred)
            except Exception as e:
                print(f"Metric {name} failed: {str(e)}")
                results[name] = np.nan
        return results

    def fit(self, X, y):
        self.results = {model[0]: [] for model in self.models}
        print(f"Name: {self.name}\n")
        for fold, (train_idx, test_idx) in enumerate(self.cv_method.split(X, y)):
            if self.verbose:
                print(f"\nFold {fold + 1}/{self.cv_method.n_splits}")
                print("-"*40)
            
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            for name, model in self.models:
                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)
                
                metrics = self._calculate_metrics(y_test, y_pred)
                self.results[name].append(metrics)
                
                if self.verbose:
                    print(f"- {name}:")
                    for m, v in metrics.items():
                        print(f"  {m}: {v:.4f}")
                    print("\n")

    def summarize(self):
        summary = {}
        for name in self.results.keys():
            fold_results = pd.DataFrame(self.results[name])
            summary[name] = {
                'mean': fold_results.mean(),
                'std': fold_results.std()
            }
        return summary

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

def macro_accuracy(y_true, y_pred):
    unique_classes = np.unique(y_true)
    acc_per_class = []
    
    for c in unique_classes:
        class_mask = y_true == c
        correct = np.sum((y_pred == y_true) & class_mask)
        acc = correct / np.sum(class_mask)
        acc_per_class.append(acc)
        
    return np.mean(acc_per_class) if acc_per_class else 0

models = [
    ("RidgeClassifier", RidgeClassifier(alpha=1.0, class_weight="balanced", random_state=CONFIG["RANDOM_STATE"])),
    # ("CosineSimClassifier", CosineSimilarityModel(threshold=0.25))
    ("LinearSVC", LinearSVC(class_weight="balanced", random_state=CONFIG["RANDOM_STATE"])),
    # ("LGBMClassifier", LGBMClassifier(verbose=-1)),
]

metric_fns = [
    ('macro_f1', f1_score),
    ('macro_accuracy', macro_accuracy),
]

cv = CrossValidator(
    models=models, 
    metric_fns=metric_fns,
    cv_method=StratifiedKFold(
        n_splits=10, shuffle=True, random_state=CONFIG["RANDOM_STATE"]), 
    name="Base Models CV"
)

cv.fit(X, y)

Name: Base Models CV


Fold 1/10
----------------------------------------
- RidgeClassifier:
  macro_f1: 0.9011
  macro_accuracy: 0.9149


- CosineSimilarity:
  macro_f1: 0.5425
  macro_accuracy: 0.5512


- LinearSVC:
  macro_f1: 0.9098
  macro_accuracy: 0.9221



Fold 2/10
----------------------------------------
- RidgeClassifier:
  macro_f1: 0.8892
  macro_accuracy: 0.9086


- CosineSimilarity:
  macro_f1: 0.4148
  macro_accuracy: 0.4334


- LinearSVC:
  macro_f1: 0.9022
  macro_accuracy: 0.9088



Fold 3/10
----------------------------------------
- RidgeClassifier:
  macro_f1: 0.9164
  macro_accuracy: 0.9344


- CosineSimilarity:
  macro_f1: 0.4733
  macro_accuracy: 0.5000


- LinearSVC:
  macro_f1: 0.9241
  macro_accuracy: 0.9380



Fold 4/10
----------------------------------------
- RidgeClassifier:
  macro_f1: 0.9124
  macro_accuracy: 0.9188


- CosineSimilarity:
  macro_f1: 0.4786
  macro_accuracy: 0.4760


- LinearSVC:
  macro_f1: 0.9054
  macro_accuracy: 0.9200



Fold 5/10

In [27]:

def view_cv_summary(results):
    for model_name, metrics in results.items():
        print(f"\n{model_name}:")
        for metric, stats in metrics['mean'].items():
            print(f"  {metric}: {stats:.4f} ± {metrics['std'][metric]:.4f}")

view_cv_summary(cv.summarize())


RidgeClassifier:
  macro_f1: 0.8970 ± 0.0335
  macro_accuracy: 0.9082 ± 0.0344

CosineSimilarity:
  macro_f1: 0.4783 ± 0.0824
  macro_accuracy: 0.5050 ± 0.0817

LinearSVC:
  macro_f1: 0.9018 ± 0.0281
  macro_accuracy: 0.9098 ± 0.0329


# Hyperparameter Tuning

In [28]:
def ridge_objective(trial, X, y, cv_method):
    params = {
        'alpha': trial.suggest_float('alpha', 1e-3, 5, log=True),
        'solver': trial.suggest_categorical('solver', ['svd', 'cholesky', 'sparse_cg']),
        'class_weight': trial.suggest_categorical('class_weight', ['balanced']),
        'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
        'random_state': trial.suggest_categorical('random_state', [CONFIG['RANDOM_STATE']])
    }
    
    model = RidgeClassifier(**params)

    fold_scores = []
    for train_idx, valid_idx in cv_method.split(X, y):
        X_train, y_train = X[train_idx], y[train_idx]
        X_valid, y_valid = X[valid_idx], y[valid_idx]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_valid)
        fold_scores.append(f1_score(y_valid, y_pred, average="macro"))
    
    return np.mean(fold_scores)

def optimize_ridge(X, y, n_trials=50):
    study = optuna.create_study(
        direction='maximize'
    )
    
    cv_method = StratifiedKFold(
        n_splits=10,
        shuffle=True,
        random_state=CONFIG["RANDOM_STATE"]
    )
    
    study.optimize(
        lambda trial: ridge_objective(trial, X, y, cv_method),
        n_trials=n_trials,
        show_progress_bar=True
    )
    
    print("Best trial:")
    trial = study.best_trial
    print(f"  Macro F1: {trial.value:.4f}")
    
    return study.best_params

best_ridge_params = optimize_ridge(X, y, n_trials=100)

[I 2025-06-25 21:17:40,388] A new study created in memory with name: no-name-11617752-9007-4588-8c95-84055cf61fc2
Best trial: 0. Best value: 0.897352:   1%|          | 1/100 [00:17<28:44, 17.42s/it]

[I 2025-06-25 21:17:57,798] Trial 0 finished with value: 0.897351962819234 and parameters: {'alpha': 0.15751050168361605, 'solver': 'svd', 'class_weight': 'balanced', 'fit_intercept': True, 'random_state': 42}. Best is trial 0 with value: 0.897351962819234.


Best trial: 0. Best value: 0.897352:   2%|▏         | 2/100 [00:36<30:09, 18.46s/it]

[I 2025-06-25 21:18:16,981] Trial 1 finished with value: 0.8913110229737828 and parameters: {'alpha': 3.2260144288094805, 'solver': 'svd', 'class_weight': 'balanced', 'fit_intercept': True, 'random_state': 42}. Best is trial 0 with value: 0.897351962819234.


Best trial: 0. Best value: 0.897352:   3%|▎         | 3/100 [00:42<20:25, 12.63s/it]

...

[I 2025-06-25 21:37:29,500] Trial 97 finished with value: 0.8947560484019824 and parameters: {'alpha': 0.10196494674689713, 'solver': 'cholesky', 'class_weight': 'balanced', 'fit_intercept': True, 'random_state': 42}. Best is trial 15 with value: 0.9005799201535603.


Best trial: 15. Best value: 0.90058:  99%|█████████▉| 99/100 [20:31<00:21, 21.19s/it]

[I 2025-06-25 21:38:12,234] Trial 98 finished with value: 0.9005799201535603 and parameters: {'alpha': 0.24054298675097616, 'solver': 'sparse_cg', 'class_weight': 'balanced', 'fit_intercept': True, 'random_state': 42}. Best is trial 15 with value: 0.9005799201535603.


Best trial: 15. Best value: 0.90058: 100%|██████████| 100/100 [21:07<00:00, 12.67s/it]

[I 2025-06-25 21:38:47,614] Trial 99 finished with value: 0.8985494716105066 and parameters: {'alpha': 0.20915902877500023, 'solver': 'sparse_cg', 'class_weight': 'balanced', 'fit_intercept': True, 'random_state': 42}. Best is trial 15 with value: 0.9005799201535603.
Best trial:
  Macro F1: 0.9006


In [45]:
def linear_svc_objective(trial, X, y, cv_method):
    params = {
        'C': trial.suggest_float('C', 1e-3, 10, log=True),
        'class_weight': trial.suggest_categorical('class_weight', ['balanced']),
        'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
        'max_iter': trial.suggest_int('max_iter', 500, 5000),
        'dual': trial.suggest_categorical('dual', [True, False]),
        'random_state': CONFIG['RANDOM_STATE']
    }
    
    model = LinearSVC(**params)

    fold_scores = []
    for train_idx, valid_idx in cv_method.split(X, y):
        X_train, y_train = X[train_idx], y[train_idx]
        X_valid, y_valid = X[valid_idx], y[valid_idx]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_valid)
        fold_scores.append(f1_score(y_valid, y_pred, average="macro"))
    
    return np.mean(fold_scores)

def optimize_linear_svc(X, y, n_trials=50):
    study = optuna.create_study(direction='maximize')
    
    cv_method = StratifiedKFold(
        n_splits=10,
        shuffle=True,
        random_state=CONFIG["RANDOM_STATE"]
    )
    
    study.optimize(
        lambda trial: linear_svc_objective(trial, X, y, cv_method),
        n_trials=n_trials,
        show_progress_bar=True
    )
    
    print("Best trial:")
    trial = study.best_trial
    print(f"  Macro F1: {trial.value:.4f}")
    print("  Params:")
    for k, v in trial.params.items():
        print(f"    {k}: {v}")
    
    return trial.params

best_linear_svc_params = optimize_linear_svc(X, y, n_trials=100)

[I 2025-06-25 21:46:27,139] A new study created in memory with name: no-name-46cd7145-8b40-4028-bc2b-043dddef1ef4
Best trial: 0. Best value: 0.859551:   1%|          | 1/100 [00:14<24:44, 14.99s/it]

[I 2025-06-25 21:46:42,128] Trial 0 finished with value: 0.8595507226568703 and parameters: {'C': 0.023994068319919824, 'class_weight': 'balanced', 'fit_intercept': True, 'max_iter': 1393, 'dual': True}. Best is trial 0 with value: 0.8595507226568703.


Best trial: 1. Best value: 0.900542:   2%|▏         | 2/100 [00:53<46:56, 28.74s/it]

[I 2025-06-25 21:47:20,497] Trial 1 finished with value: 0.9005421758009398 and parameters: {'C': 0.9208203813204336, 'class_weight': 'balanced', 'fit_intercept': False, 'max_iter': 2310, 'dual': True}. Best is trial 1 with value: 0.9005421758009398.


Best trial: 1. Best value: 0.900542:   3%|▎         | 3/100 [01:38<58:51, 36.40s/it]

...

[I 2025-06-25 22:35:50,457] Trial 97 finished with value: 0.9035275961445798 and parameters: {'C': 0.4517820524428648, 'class_weight': 'balanced', 'fit_intercept': False, 'max_iter': 4889, 'dual': False}. Best is trial 31 with value: 0.9044086678715162.


Best trial: 31. Best value: 0.904409:  99%|█████████▉| 99/100 [49:49<00:26, 26.77s/it]

[I 2025-06-25 22:36:16,574] Trial 98 finished with value: 0.9026867613740966 and parameters: {'C': 0.7719514374319031, 'class_weight': 'balanced', 'fit_intercept': True, 'max_iter': 4207, 'dual': False}. Best is trial 31 with value: 0.9044086678715162.


Best trial: 31. Best value: 0.904409: 100%|██████████| 100/100 [50:16<00:00, 30.17s/it]

[I 2025-06-25 22:36:44,078] Trial 99 finished with value: 0.9022199961815177 and parameters: {'C': 0.28044215886217616, 'class_weight': 'balanced', 'fit_intercept': True, 'max_iter': 4710, 'dual': False}. Best is trial 31 with value: 0.9044086678715162.
Best trial:
  Macro F1: 0.9044
  Params:
    C: 0.513724142614831
    class_weight: balanced
    fit_intercept: True
    max_iter: 3530
    dual: True


In [57]:
op_models = [
    ("RidgeClassifier", RidgeClassifier(**best_ridge_params)),
    ("LinearSVC", LinearSVC(**best_linear_svc_params))
]

op_cv = CrossValidator(
    models=op_models + [("Ridge+LinearSVC Stack", VotingClassifier([*op_models], weights=[0.2, 0.8]))],
    metric_fns=metric_fns,
    cv_method=StratifiedKFold(
        n_splits=10, shuffle=True, random_state=CONFIG["RANDOM_STATE"]), 
    name="FaceRecognition CV"
)

op_cv.fit(X, y)

Name: FaceRecognition CV


Fold 1/10
----------------------------------------
- RidgeClassifier:
  macro_f1: 0.9091
  macro_accuracy: 0.9221


- LinearSVC:
  macro_f1: 0.9039
  macro_accuracy: 0.9181


- Ridge+LinearSVC Stack:
  macro_f1: 0.9039
  macro_accuracy: 0.9181



Fold 2/10
----------------------------------------
- RidgeClassifier:
  macro_f1: 0.8890
  macro_accuracy: 0.9086


- LinearSVC:
  macro_f1: 0.9226
  macro_accuracy: 0.9231


- Ridge+LinearSVC Stack:
  macro_f1: 0.9226
  macro_accuracy: 0.9231



Fold 3/10
----------------------------------------
- RidgeClassifier:
  macro_f1: 0.9184
  macro_accuracy: 0.9321


- LinearSVC:
  macro_f1: 0.9216
  macro_accuracy: 0.9380


- Ridge+LinearSVC Stack:
  macro_f1: 0.9216
  macro_accuracy: 0.9380



Fold 4/10
----------------------------------------
- RidgeClassifier:
  macro_f1: 0.9203
  macro_accuracy: 0.9224


- LinearSVC:
  macro_f1: 0.9053
  macro_accuracy: 0.9200


- Ridge+LinearSVC Stack:
  macro_f1: 0.9053
  macro_accur

In [58]:
view_cv_summary(op_cv.summarize())


RidgeClassifier:
  macro_f1: 0.9006 ± 0.0296
  macro_accuracy: 0.9108 ± 0.0302

LinearSVC:
  macro_f1: 0.9044 ± 0.0269
  macro_accuracy: 0.9133 ± 0.0271

Ridge+LinearSVC Stack:
  macro_f1: 0.9044 ± 0.0269
  macro_accuracy: 0.9133 ± 0.0271


# Predicting on Test set

In [39]:
def create_test_data(test_dir, app, enhanced_embedding=False):
    test_data = {
        "embedding": [],
        "enhanced_embedding": [],
        "norm_bbox": [],
        "filenames": []
    }
    
    failed_samples = []
    img_path = os.path.join(test_dir, "images")
    for filename in tqdm(sorted(os.listdir(img_path)), desc="Processing Test Identites"):
        filepath = os.path.join(img_path, filename)
        
        img = cv2.imread(filepath)
        faces = robust_face_detection(img, app, attempts=4)

        if faces:
            face = faces[0]
            img_h, img_w = img.shape[:2]
            bbox = face.bbox
            norm_bbox = [
                bbox[0]/img_w,
                bbox[1]/img_h,
                bbox[2]/img_w,
                bbox[2]/img_h
            ]
            test_data['embedding'].append(
                (face.embedding / np.linalg.norm(face.embedding))
            )
            test_data['enhanced_embedding'].append(
                enhance_embedding(face.embedding, face)
            )
            test_data['norm_bbox'].append(norm_bbox)
        else:
            test_data['embedding'].append(np.zeros(512))
            test_data['enhanced_embedding'].append(np.zeros(738))
            test_data['norm_bbox'].append([-1.0, -1.0, -1.0, -1.0])
            failed_samples.append((filepath))
        test_data['filenames'].append(filename)

    for key in test_data:
        if key == 'filenames':
            continue
        test_data[key] = np.array(test_data[key])
    
    return test_data, failed_samples

test_data, test_failed = create_test_data(test_dir, app, enhanced_embedding=CONFIG["ENHANCE_EMB"])

Processing Test Identites: 100%|██████████| 636/636 [03:44<00:00,  2.83it/s]


In [40]:
len(test_failed)

191

In [41]:
test_data['embedding'].shape, test_data['enhanced_embedding'].shape

((636, 512), (636, 738))

In [42]:
test_feats, _, test_metadata = create_feature_vectors(
    test_data, scaler, weights=CONFIG["FEATURE_WEIGHTS"], enhanced_embedding=CONFIG["ENHANCE_EMB"], return_target=False
)
test_feats.shape

(636, 742)

In [44]:
ridge = RidgeClassifier(**best_ridge_params)
ridge.fit(X, y)

ridge_preds = ridge.predict(test_feats)
ridge_preds[:20]

array([19, 14, 16, 18,  2, 19, 14,  9, 18,  4, 14, 18,  1, 14, 18, 18, 18,
        2, 16, 18])

In [45]:
submission = pd.DataFrame(columns=["filename", "employee_id"])
submission["filename"] = test_data['filenames']
submission["employee_id"] = le.classes_[ridge_preds]

submission.head(15)

,filename,employee_id
0,face_0002.jpg,emp019
1,face_0005.jpg,emp014
2,face_0009.jpg,emp016
3,face_0012.jpg,emp018
4,face_0014.jpg,emp002
5,face_0015.jpg,emp019
6,face_0017.jpg,emp014
7,face_0018.jpg,emp009
8,face_0020.jpg,emp018
9,face_0021.jpg,emp004


In [46]:
submission.to_csv("submission.csv", index=False)